# Progetto Big Data in Geographic Information Systems

Analyze and compare two geospatial datasets  
The choice of the datasets is free, but a few constraints must be observed.  
	• Use at least 2 different datasets (e.g. model and observations, n different models, 2 different observational datasets, etc.)  
	• Perform at least 1 operation in space and 1 operation in time (e.g. aggregation, normalization, resampling, regridding, etc.)  
	• Display data both as a function of space and time (time series and maps)  
	• Apply some sort of statistical analysis on the data (arrange the data according to some pdf, hypothesis testing, time series analysis, detection of trends, classification, pattern detection, ...)

# Indice
* [Setting](#setting)
* [Lettura dataset](#lettura-dataset)
    * [Dataset Sea Surface Temperature](#dataset-sst)
    * [Dataset Ice Concentration](#dataset-icec)
* [Esplorazione dataset Sea Surface Temperature](#espl-sst)
    * [Caratteristiche principali](#car-pr-sst)
    * [Distribuzione della variabile sst](#dist-sst)
    * [Confronto gennaio 1982 - gennaio 2022](#gennaio-sst)
    * [Sea Surface Temperature Anomaly map](#anom-map-sst)
    * [Sea Surface Temperature Anomaly](#anom-sst)
    * [Focus sulle aree polari](#aree-polari-sst)
    * [Analisi serie storiche](#serie-sst)
* [Esplorazione dataset Ice Concentration](#espl-icec)
    * [Caratteristiche principali](#car-pr-icec)
    * [Distribuzione della variabile icec](#dist-icec)
    * [Confronto gennaio 1982 - gennaio 2022](#gennaio-icec)
    * [Ice Concentration Anomaly map](#anom-map-icec)
    * [Comparazione Northen e Southern Hemisphere Ice Concentration Anomaly](#emis)
    * [Focus sulle aree polari](#aree-polari-icec)
    * [Analisi serie storiche](#serie-icec)
      * [Polo Nord](#nord)
      * [Polo Sud](#sud)
* [Sea Surface Temperature data e Ice Concentration data](#correlation-data)
    * [Global Sea Surface Temperature e Ice Concentration Anomaly](#global)
    * [Correlazione tra Sea Surface Temperature e Ice Concentration](#correlazione)

# Setting <a class="anchor" id="setting"></a>

In [ ]:
import numpy as np
import seaborn as sns
import cmocean
import cartopy.crs as ccrs
from matplotlib import pyplot as plt
import cartopy.feature as cfeature
import xarray as xr
import matplotlib as mpl
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
import pandas as pd
from numpy.random import seed
from scipy.stats import pearsonr
from numpy.random import randn
import hvplot.xarray
import datetime
import warnings
from IPython.display import Image
seed(1)

%matplotlib inline
%config InlinedBackend.figure_format = 'retina'
warnings.simplefilter(action='ignore')

# Lettura dataset <a class="anchor" id="lettura-dataset"></a>

La National Oceanic and Atmospheric Administration (NOAA) è una agenzia federale statunitense che si interessa di oceanografia, meteorologia e climatologia.

La Ricerca Oceanica e Atmosferica (OAR o NOAA Research) è una divisione della National Oceanic and Atmospheric Administration.

Tra i dieci laboratori di ricerca della OAR vi è il NOAA Physical Sciences Laboratory (PSL).

Il NOAA PSL ha collezionato e reso disponibili i file netCDF (a livello settimanale e mensile) NOAA OI SST V2 (NOAA Optimum Interpolation Sea Surface Temperature V2).

I dati considerati nel progetto sono a granularità mensile.

I dati sono relativi a temperature superficiali del mare e a percentuali di concentrazione di ghiaccio nelle aree oceaniche. 

Con "temperatura superficiale" del mare si intende da 1 mm a 20 m sotto il livello della superficie del mare, a seconda del metodo di misurazione utilizzato.  I valori di tale temperatura sono molto rilevanti dal momento in cui le masse d'aria nell'atmosfera terrestre sono fortemente influenzate dalla temperatura della superficie del mare che viene cosi a costituire una cosidetta "forzante" della circolazione atmosferica.

Le aree oceaniche coperte parzialmente di ghiaccio si riferiscono alle porzioni di ghiaccio che fluttuano intorno alle regioni dell'antartide e della regione antartica. Questo ghiaccio interagisce con molti sistemi globali, tra cui l'ecosistema globale. Interagisce anche con il sistema climatico, mantenendo fresche le regioni polari. Non da ultimo queste porzioni di ghiaccio interagiscono anche con gli oceani, per mezzo dei quali vi è una ridistribuzione del calore intorno al globo.

info:  
https://psl.noaa.gov/data/gridded/data.noaa.oisst.v2.html  
https://arctic.noaa.gov/Report-Card

## Dataset Sea Surface Temperature <a class="anchor" id="dataset-sst"></a>

In [ ]:
sst_data = xr.open_dataset('sst.mnmean.nc')  # sea_surface_temperature (degC) - sst

# viene applicata una mask per le zone di terra ferma
dmask  = xr.open_dataset('lsmask.nc')
sst_data['sst'] = sst_data.sst.where(dmask.mask.isel(time=0) == 1)

# viene modificata la griglia
sst_data.coords['lon'] = (sst_data.coords['lon'] + 180) % 360 - 180
sst_data = sst_data.sortby(sst_data.lon)
sst_data

## Dataset Ice Concentration <a class="anchor" id="dataset-icec"></a>

In [ ]:
icec_data = xr.open_dataset('icec.mnmean.nc')  # Ice Concentration Mean Surface (%) - icec

# viene modificata la griglia
icec_data.coords['lon'] = (icec_data.coords['lon'] + 180) % 360 - 180
icec_data = icec_data.sortby(icec_data.lon)
icec_data

# Esplorazione dataset Sea Surface Temperature <a class="anchor" id="espl-sst"></a>

## Caratteristiche principali <a class="anchor" id="car-pr-sst"></a>

In [ ]:
sst_data.attrs

In [ ]:
sst_data.sst.min()  # -1.79
sst_data.sst.max()  # 35.56

Si hanno i dati della temperatura superificiale del mare con granularità mensile.

Viene utilizzato il primo giorno del mese alle ore 00.00.00 per indicare il label del mese.

In particolare si hanno i dati mensili da dicembre 1981 ad aprile 2022, per un totale di 485 istanti temporali.

La griglia è composta da 180 bande di latitudine (ovvero 180 valori di latitudine) e 360 bande di longitudine.
I valori delle variabili di latitudine e longitudine rappresentano i centroidi delle celle che compongono la griglia.

## Distribuzione della variabile sst <a class="anchor" id="dist-sst"></a>

In [ ]:
# sst_data.sst.plot()
sst_data.sst.hvplot()

# un elevato numero di osservazioni indicano una temperatura superficiale dell'acqua prossima allo zero

In [ ]:
# grafico interattivo per osservare l'andamento della temperatura annualmente a partire dal 1981

proj = ccrs.PlateCarree()
sst_data.sst.isel(time=slice(0, 485, 12)).hvplot.quadmesh('lon',
                                                          'lat',
                                                          crs = ccrs.PlateCarree(),
                                                          projection=proj,
                                                          project=True,
                                                          global_extent=True,
                                                          rasterize=True,
                                                          color='seawater_temperature',
                                                          cmap=cmocean.cm.thermal,
                                                          dynamic=False,
                                                          coastline=True,
                                                          frame_width=500,
                                                          clim = (-1, 35),
                                                          figsize=(10,13))

# si notano alcune variazioni, in particolar modo nelle zone temperate e tropicali

In [ ]:
# temperature medie per ciascun mese

sst = sst_data.sst.groupby('time.month').mean('time')  # medie di tutti i mesi per tutti gli anni

# grafico interattivo
proj = ccrs.PlateCarree()
sst.isel(month=slice(0, 12, 1)).hvplot.quadmesh('lon',
                                                'lat',
                                                crs = ccrs.PlateCarree(),
                                                projection=proj,
                                                project=True,
                                                global_extent=True,
                                                rasterize=True,
                                                color='seawater_temperature',
                                                cmap=cmocean.cm.thermal,
                                                dynamic=False,
                                                coastline=True,
                                                frame_width=500,
                                                clim = (-1, 35),
                                                figsize=(10,13))

# si notano alcune variazioni, in particolar modo nelle zone temperate e tropicali

In [ ]:
# si comprime la dimensione del tempo calcolando la media, viene dunque riportata la media della variabile sst lungo la linea del tempo

fig = plt.figure(figsize=(14,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND,zorder=4,color='white')  # al posto di usare il dataset mask, si ottiene una visualizzazione più accurata
ax.gridlines(draw_labels=True)
sst_data.sst.mean(axis=0).plot(cmap=cmocean.cm.thermal,
                               vmin=-1,
                               vmax=35)

# si osservano come prevedibile temperature elevate nei pressi dell'equatore e prossime allo zero nelle aree polari

## Confronto gennaio 1982 - gennnaio 2022 <a class="anchor" id="gennaio-sst"></a>

In [ ]:
# si mette a confronto il primo mese di gennaio con l'ultimo corrispondente per vedere se sono visibili graficamente delle differenze

fig = plt.figure(figsize=(20,8))
subplots = (1,2)
n_panels = subplots[0] * subplots[1]


# gennaio 1982
sst = sst_data.sel(time='1982-01-01', method='nearest')
ax = fig.add_subplot(subplots[0], subplots[1], 1, projection=ccrs.Robinson())
ax.add_feature(cfeature.LAND,zorder=4, color='white')  
# ax.set_title('January 1982')
ax.set_global()
mm = ax.pcolormesh(sst.lon,
                   sst.lat,
                   sst.sst,
                   transform=ccrs.PlateCarree(),
                   vmin=-1,
                   vmax=35,
                   cmap=cmocean.cm.thermal)
ax.coastlines()
ax.gridlines(draw_labels=True)


# gennaio 2022
sst = sst_data.sel(time='2022-01-01', method='nearest')
# ax.set_title('January 2022')
ax = fig.add_subplot(subplots[0], subplots[1], 2, projection=ccrs.Robinson())
ax.add_feature(cfeature.LAND,zorder=4, color='white')
ax.set_global()
mm = ax.pcolormesh(sst.lon,
                   sst.lat,
                   sst.sst,
                   transform=ccrs.PlateCarree(),
                   vmin=-1,
                   vmax=35,
                   cmap=cmocean.cm.thermal)
ax.coastlines()
ax.gridlines(draw_labels=True)


cbar_ax = fig.add_axes([0.28, 0.10, 0.46, 0.05])
cbar = fig.colorbar(mm, cax=cbar_ax, extend='both', orientation='horizontal')
cbar.set_label('Celsius')
cbar.ax.tick_params(labelsize=8)


plt.show()
plt.close()


# come prevedibile, trattandosi di un arco temporale ridotto, osservando i due grafici la differenza non è netta. Si potrebbe ipotizzare 
# che nei pressi dell'equatore si veda un incremento delle temperature con il passare degli anni

# in ogni caso da questi plot si può notare, come osservato precedentemente, che le temperature più basse si verificano nei pressi dei 
# poli mentre avvicinandosi all'equatore le temperature aumentano in modo graduale

## Sea Surface Temperature Anomaly map <a class="anchor" id="anom-map-sst"></a>

In [ ]:
# Extract time slices
first = sst_data.sel(time=slice("1981-01", "2010-12"))  # periodo di riferimento
last = sst_data.sel(time=slice("2022-03"))  # si prende come mese marzo 2022
diff = last.sst.mean(axis=0)-first.sst.mean(axis=0)

# Set plot
fig = plt.figure(figsize=(8,6)) 

subplots = (1,1)
n_panels = subplots[0] * subplots[1]

tmax = np.abs(diff).max()
norm = mpl.colors.Normalize(vmin=-tmax,vmax=tmax)
cmap = mpl.cm.seismic

ax = fig.add_subplot(subplots[0], subplots[1], 1, projection=ccrs.Robinson())
ax.set_global()
mm = ax.pcolormesh(sst_data.lon, sst_data.lat, diff,
                   transform=ccrs.PlateCarree(),cmap=cmap, norm=norm )
ax.coastlines()
ax.gridlines(draw_labels=True)

# add colorbar
cbar_ax = fig.add_axes([0.28, 0.10, 0.46, 0.05]) #[left, bottom, width, height]
cbar = fig.colorbar(mm, cax=cbar_ax, extend='both', orientation='horizontal')
cbar.set_label('Celsius')
cbar.ax.tick_params(labelsize=8)

plt.show()
plt.close()

# come si può osservare in gran parte del globo si osservano delle anomalie positive, quindi delle temperature più elevate rispetto 
# al periodo di riferimento. Questo indica un riscaldamento globale delle temperature superficiali dei mari

# l'artico si sta riscaldando fino a due/tre volte più velocemente rispetto al resto del globo ed è considerato estremamente 
# vulnerabile ai cambiamenti climatici, contribuendo all'innalzamento del livello dei mari con la fusione della Groenlandia e dei ghiacciai

# nei pressi dell'antartide si notano delle anomalie negative, ciò che rende il riscaldamento antartico complesso è che il continente meridionale, 
# ad eccezione della sua penisola che si sta riscaldando rapidamente e che sta perdendo rapidamente ghiaccio, non si è riscaldato molto, 
# soprattutto se confrontato con il resto del globo

# per cercare di comprendere al meglio le tendenze del clima è infatti perferibile considerare il polo nord come indicatore climatico 
# https://earthdata.nasa.gov/learn/sensing-our-planet/unexpected-ice

## Sea Surface Temperature Anomaly <a class="anchor" id="anom-sst"></a>

In [ ]:
ds = sst_data
ds = ds.sst.weighted(weights).mean(dim=['lat','lon'])  # media peata
enso_clim = ds.sel(time=slice('1982-01-01','2010-12-01')).groupby('time.month').mean('time')
ds_mean = ds.groupby('time.month') - enso_clim

ds_mean.plot(figsize=(12,6), c='black')
plt.axhline(y=0, c='black', linewidth=1, linestyle='dashdot')
plt.ylabel('Sea Surface Temperature anomaly')
time = ds_mean.time.values
plt.xlim([datetime.date(1981,12,1), datetime.date(2022,5,1)])
plt.title('Sea Surface Temperature anomaly')

# è possibile osservare a partire agli anni 2000 delle anomalie in gran parte positive

# https://xarray.pydata.org/en/stable/examples/weather-data.html#Calculate-monthly-anomalies

## Focus sulle aree polari <a class="anchor" id="aree-polari-sst"></a>
Tra il 14 e il 20 marzo 2022 due eventi meteorologici eccezionali hanno interessato Polo Nord e Polo Sud. Sarebbe più corretto dire che, in particolare, l'evento che ha interessato l’Antartide ha ridefinito la storia climatologica del continente: un evento ritenuto inimmaginabile fino a poco tempo fa. 

- https://polarjournal.ch/en/2022/03/22/heat-waves-over-polar-regions-baffle-experts
- http://www.lamma.rete.toscana.it/news/dallartide-allantartide-eventi-senza-precedenti
- https://www.repubblica.it/green-and-blue/2022/03/23/news/il_caldo_record_ai_poli_che_allarma_gli_scienziati-342549265
  
In Antartide si sono registrate temperature anche 40 gradi sopra la media (tra il 17 e il 20 marzo su un’ampia zona di Antartide orientale si è osservato un marcato e rapido aumento delle temperature).  

*Extraordinary anomalies in #Antarctica lead to historic records today, 18 march 2022:  
-Vostok  3489m -17.7C,monthly record beaten by nearly 15C !  
-Concordia 3234m -12.2C,highest Temp. on records  and about 40C above average !  
-Dome C II 3250m -10.1C  
-D-47 1560m -3.3C  
-Terra Nova Base 74S +7.0C*

In [ ]:
# polo nord

# marzo medio
sst = sst_data.sst.groupby('time.month').mean('time')
sst_mar = sst.isel(month=2)  # 2 corrisponde a marzo
fig = plt.figure(figsize=(10,6))
extent = [180, -180 , 50 , 90]  
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = sst_mar.plot(transform=ccrs.PlateCarree(),
                                                       vmin=-1,
                                                       vmax=35,
                                                       cmap=cmocean.cm.thermal)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()


# marzo 2022
fig = plt.figure(figsize=(10,6))
extent = [180, -180 , 50 , 90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = sst_data.sst.isel(time=483).plot(transform=ccrs.PlateCarree(),
                                                       vmin=-1,
                                                       vmax=35,
                                                       cmap=cmocean.cm.thermal)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()

# si utilizza Orthographic Projection
# https://www.winwaed.com/blog/2010/01/11/polar-maps-and-projections-part-1-overview/

In [ ]:
# polo sud

# in modo analogo si prende in considerazione il mese di marzo per il polo sud

# marzo medio
sst = sst_data.sst.groupby('time.month').mean('time')
sst_mar = sst.isel(month=2)  # 2 corrisponde a marzo
fig = plt.figure(figsize=(10,6))
extent = [180, -180 , -50 , -90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=-90))  
ax.set_extent(extent, ccrs.PlateCarree())
p1a = sst_mar.plot(transform=ccrs.PlateCarree(),
                                                       vmin=-1,
                                                       vmax=35,
                                                       cmap=cmocean.cm.thermal)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()


# marzo 2022
fig = plt.figure(figsize=(10,6))
extent = [180, -180 , -50 , -90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=-90)) 
ax.set_extent(extent, ccrs.PlateCarree())
p1a = sst_data.sst.isel(time=483).plot(transform=ccrs.PlateCarree(),
                                                       vmin=-1,
                                                       vmax=35,
                                                       cmap=cmocean.cm.thermal)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()

# tuttavia in questo caso come per il polo nord non si percepisono differenze nette anche perchè si stanno considerando temperature 
# superficiali dei mari mentre quelle misurate, menzionate negli articoli citati precedentemente, sono temperature misurate in superficie

## Analisi serie storiche <a class="anchor" id="serie-sst"></a>

In [ ]:
# visualizzazione serie storica della variabile sst a diverse granularità: mensile e annuale

plt.figure(figsize=(20, 6))
weights = np.cos(np.deg2rad(sst_data.sst.lat))  # su una griglia lat e long le aree delle celle sono diverse, tendono a diminuire verso i poli
# risulta dunque necessario effettuare una media pesata con i pesi che siano proporzionali alle aree delle celle. Si può usare ad esempio la funzione 
# coseno per creare questi pesi
weights.name = "weights"
sst_weighted = sst_data.sst.weighted(weights)
weighted_mean = sst_weighted.mean(("lon", "lat"))

weighted_mean.plot()  # serie temporale mensile globale con media pesata

weighted_mean.resample(time="Y").mean().plot()  # serie temporale annuale globale con media pesata

plt.legend(["monthly time series", "annual time series"], loc=0)
plt.show()

# come si vede chiaramente si ha un trend crescente nel tempo, il che sta ad indicare che negli ultimi anni la temperatura superficiale 
# dei mari è aumentata

# volendo quantificare questo riscaldamento degli oceani, negli ultimi 40 anni, dal 1981 ad oggi, è possibile osservare che la temperatura 
# superficiale media degli oceani si è alzata di +0,8°C
# http://meteobook.it/temperature-del-mare-guardate-come-sono-cambiate-negli-ultimi-40-anni/

Concentrandosi sulle aree polari:  
Per le analisi successive si è deciso di considerare per il polo nord e il polo sud le rispettive regioni polari.  
Polo nord: dal circolo polare artico (lat: 66.5 circa) alla latitudine 89.5 (primi 24 valori di latitudine)  
Polo sud: dal circolo polare antartico (lat: -66.5 circa) alla latitudine -89.5 (ultimi 24 valori di latitudine) 

In [ ]:
Image(filename='terra.gif') 

In [ ]:
# polo nord

# sst_data.isel(lat=slice(0,24)).mean(("lon", "lat"), keep_attrs=True).sst.hvplot()
sst_data.isel(lat=slice(0,24)).sst.weighted(weights).mean(("lon", "lat"), keep_attrs=True).hvplot()  # media pesata

# la temperatura oscilla tra -1 grado centigrado e i 2 gradi. Negli ultimi anni la temperatura si è avvicinata ai 3 gradi nei mesi di agosto 

In [ ]:
# polo sud

# sst_data.isel(lat=slice(156,180)).mean(("lon", "lat"), keep_attrs=True).sst.hvplot()
sst_data.isel(lat=slice(156,180)).sst.weighted(weights).mean(("lon", "lat"), keep_attrs=True).hvplot()  # media pesata

# mentre nell'area del polo nord si può vedere un accenno di trend positivo, considerando la serie storica relativa all'area del polo sud ciò 
# è meno marcato
# la variabilità sembra essere maggiore per il polo nord

In [ ]:
# andamento delle temperature per polo nord e polo sud che rispecchia il fenomeno delle stagioni invertite

# senza media pesata
# plt.figure(figsize=(20, 6))
# sst_data.isel(lat=slice(0,24)).mean(("lon", "lat"), keep_attrs=True).sst.groupby("time.month").mean("time").plot()  
# sst_data.isel(lat=slice(156,180)).mean(("lon", "lat"), keep_attrs=True).sst.groupby("time.month").mean("time").plot()  
# plt.legend(["north pole area", "south pole area"], loc=0)
# plt.show()

# con media pesata
plt.figure(figsize=(20, 6))
sst_data.isel(lat=slice(0,24)).sst.weighted(weights).mean(("lon", "lat"), keep_attrs=True).groupby("time.month").mean("time").plot()  
sst_data.isel(lat=slice(156,180)).sst.weighted(weights).mean(("lon", "lat"), keep_attrs=True).groupby("time.month").mean("time").plot()  
plt.legend(["north pole area", "south pole area"], loc=0)
plt.show()

# correttamente si vede che le stagioni sono invertite per i due emisferi

# Esplorazione dataset Ice Concentration <a class="anchor" id="espl-icec"></a>

## Caratteristiche principali <a class="anchor" id="car-pr-icec"></a>

In [ ]:
icec_data.attrs

In [ ]:
icec_data.icec.min()  # 0
icec_data.icec.max()  # 100

# si tratta di valori in percentuale
# 0 è diverso da "nan", 0 indica che non vi è ghiaccio mentre "nan" indica che si tratta di terra ferma

Si hanno i dati della concentrazione di ghiaccio con granularità mensile. Vi sono valori null in corrispondenza della terra ferma.

Viene utilizzato il primo giorno del mese alle ore 00.00.00 per indicare il label del mese.

In particolare si hanno i dati mensili da dicembre 1981 ad aprile 2022, per un totale di 484 istanti temporali.

La griglia è composta da 180 bande di latitudine e 360 bande di longitudine.

## Distribuzione della variabile icec <a class="anchor" id="dist-icec"></a>

In [ ]:
# icec_data.icec.plot()
icec_data.icec.hvplot()

In [ ]:
# grafico interattivo per vedere l'andamento delle percentuali di concentrazione dei ghiacci annualmente a partire dal 1981

proj = ccrs.PlateCarree()
icec_data.icec.isel(time=slice(0, 485, 12)).hvplot.quadmesh('lon',
                                                         'lat',
                                                         crs = ccrs.PlateCarree(),
                                                         projection=proj,
                                                         project=True,
                                                         global_extent=True,
                                                         rasterize=True,
                                                         color='seawater_temperature',
                                                         cmap=cmocean.cm.ice,
                                                         dynamic=False,
                                                         coastline=True,
                                                         frame_width=500,
                                                         clim = (0, 100),
                                                         figsize=(10,13))

In [ ]:
# percentuali di concentrazione di ghiaccio medie per ciascun mese

icec = icec_data.icec.groupby('time.month').mean('time')  # medie di tutti i mesi per tutti gli anni

# grafico interattivo
proj = ccrs.PlateCarree()
icec.isel(month=slice(0, 12, 1)).hvplot.quadmesh('lon',
                                                         'lat',
                                                         crs = ccrs.PlateCarree(),
                                                         projection=proj,
                                                         project=True,
                                                         global_extent=True,
                                                         rasterize=True,
                                                         color='ice',
                                                         cmap=cmocean.cm.ice,
                                                         dynamic=False,
                                                         coastline=True,
                                                         frame_width=500,
                                                         clim = (0, 100),
                                                         figsize=(10,13))


# le aree in mezzo ai continenti rappresentano laghi, mar rosso ecc.

In [ ]:
# si comprime la dimensione del tempo calcolando la media
# viene riportata la media della variabile icec lungo la linea del tempo

fig = plt.figure(figsize=(14,8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND,zorder=4,color='white')
ax.gridlines(draw_labels=True)
icec_data.icec.mean(axis=0).plot(cmap=cmocean.cm.ice,
                                 vmin=0,
                                 vmax=100)

## Confronto gennaio 1982 - gennaio 2022 <a class="anchor" id="gennaio-icec"></a>

In [ ]:
# si mette a confronto il primo mese di gennaio con l'ultimo corrispondente per vedere se sono visibili graficamente delle differenze

fig = plt.figure(figsize=(20,8))
subplots = (1,2)
n_panels = subplots[0] * subplots[1]


# gennaio 1982
icec = icec_data.sel(time='1982-01-01', method='nearest')
ax = fig.add_subplot(subplots[0], subplots[1], 1, projection=ccrs.Robinson())
ax.add_feature(cfeature.LAND,zorder=4,color='white')  
# ax.set_title('January 1982')
ax.set_global()
mm = ax.pcolormesh(icec.lon,
                   icec.lat,
                   icec.icec,
                   transform=ccrs.PlateCarree(),
                   vmin=0,
                   vmax=100,
                   cmap=cmocean.cm.ice)
ax.coastlines()
ax.gridlines(draw_labels=True)



# gennaio 2022
icec = icec_data.sel(time='2022-01-01', method='nearest')
# ax.set_title('January 2022')
ax = fig.add_subplot(subplots[0], subplots[1], 2, projection=ccrs.Robinson())
ax.add_feature(cfeature.LAND,zorder=4,color='white')  # al posto di usare il dataset mask
ax.set_global()
mm = ax.pcolormesh(icec.lon,
                   icec.lat,
                   icec.icec,
                   transform=ccrs.PlateCarree(),
                   vmin=0,
                   vmax=100,
                   cmap=cmocean.cm.ice)
ax.coastlines()
ax.gridlines(draw_labels=True)


cbar_ax = fig.add_axes([0.28, 0.10, 0.46, 0.05])
cbar = fig.colorbar(mm, cax=cbar_ax, extend='both', orientation='horizontal')
cbar.set_label('Percentage')
cbar.ax.tick_params(labelsize=8)


plt.show()
plt.close()

# si notano dei cambiamenti nella distribuzione dei ghiacci sia al polo nord che al polo sud. Si indaga più nel dettaglio con le analisi 
# successive. Le differenze sono più nette rispetto al dataset precedente

## Ice Concentration Anomaly map <a class="anchor" id="anom-map-icec"></a>

In [ ]:
# Extract time slices
first = icec_data.sel(time=slice("1981-01", "2010-12"))  # periodo di riferimento
last = icec_data.sel(time=slice("2022-03"))  # si prende come mese marzo 2022
diff = last.icec.mean(axis=0)-first.icec.mean(axis=0)

# Set plot
fig = plt.figure(figsize=(8,6))  # x,y(inches)

subplots = (1,1)
n_panels = subplots[0] * subplots[1]

tmax = np.abs(diff).max()
norm = mpl.colors.Normalize(vmin=-tmax,vmax=tmax)
cmap = mpl.cm.seismic

ax = fig.add_subplot(subplots[0], subplots[1], 1, projection=ccrs.Robinson())
ax.set_global()
mm = ax.pcolormesh(icec_data.lon, icec_data.lat, diff,
                   transform=ccrs.PlateCarree(),cmap=cmap, norm=norm )
ax.coastlines()
ax.gridlines(draw_labels=True)

#- add colorbar
cbar_ax = fig.add_axes([0.28, 0.10, 0.46, 0.05]) #[left, bottom, width, height]
cbar = fig.colorbar(mm, cax=cbar_ax, extend='both', orientation='horizontal')
cbar.set_label('Percentage')
cbar.ax.tick_params(labelsize=8)

plt.show()
plt.close()

# come si può osservare nel 2022 rispetto ai valori di riferimento relativi al periodo 1981-2010 si ha una netta diminuzione dei ghiacci 
# nella zona polare artica. Questa discostamento negativo rispetto alla media è di portata minore per la 
# zona polare antartica. Infatti in antartide ci sono diversi fattori che contribuiscono alla formazione dei ghiacci in maggior presenza
# rispetto al polo nord. Tra questi fattori si distinguono: il buco dell'ozono sopra l'antartide permette di raffreddare la stratosfera e di creare
# un vortice polare più forte di quello misurato al polo nord, il forte vento porta alla formazione di polynyas, aree in cui è più facile 
# che si possa formare ulteriore ghiaccio e non da ultimo la circolazione oceanica.
# https://climate.copernicus.eu/sea-ice-cover-july-2021
# https://skepticalscience.com/antarctica-gaining-ice-intermediate.htm

## Comparazione Northen e Southern Hemisphere Ice Concentration Anomaly <a class="anchor" id="emis"></a>

In [ ]:
# Arctic Sea Ice Concentration anomaly
ds = icec_data.sel(lat=slice(90, 0))
ds = ds.icec.weighted(weights).mean(dim=['lat','lon'])  # media peata
enso_clim = ds.sel(time=slice('1982-01-01','2010-12-01')).groupby('time.month').mean('time')
ds_mean = ds.groupby('time.month') - enso_clim

ds_mean.plot(figsize=(12,6), c='black')
plt.axhline(y=0, c='black', linewidth=1, linestyle='dashdot')
plt.ylabel('Arctic Sea Ice concentration anomaly')
time = ds_mean.time.values
plt.xlim([datetime.date(1981,12,1), datetime.date(2022,5,1)])
plt.title('Arctic Sea Ice concentration anomaly')
# il grafico mostra anomalie per lo più positive dal 1981 al 2004. Dal 2005 si osservano delle anomalie negative
# questa osservazione verrà confermata successivamente con i risultati ottenuti applicando il modello Prophet

# per il polo nord si ha una variabilità più elevata 



# Antarctic Sea Ice Concentration anomaly
ds = icec_data.sel(lat=slice(0, -90))
ds = ds.icec.weighted(weights).mean(dim=['lat','lon'])  # media peata
enso_clim = ds.sel(time=slice('1982-01-01','2010-12-01')).groupby('time.month').mean('time')
ds_mean = ds.groupby('time.month') - enso_clim

ds_mean.plot(figsize=(12,6), c='black')
plt.axhline(y=0, c='black', linewidth=1, linestyle='dashdot')
plt.ylabel('Antarctic Sea Ice concentration anomaly')
time = ds_mean.time.values
plt.xlim([datetime.date(1981,12,1), datetime.date(2022,5,1)])
plt.title('Antarctic Sea Ice concentration anomaly')
# il grafico mostra un numero maggiore di anomalie negative. Tra il 2007 e il 2015 vi è un periodo in cui la concentrazione di ghiaccio 
# è aumentata


# tali risultati coincidono con le osservazioni presenti in:
# https://climate.copernicus.eu/sea-ice-cover-october-2021  
# https://climate.copernicus.eu/sea-ice-cover-july-2021

## Focus sulle aree polari <a class="anchor" id="aree-polari-icec"></a>
La regione antartica è rappresentata da un oceano ricoperto da un sottile e molto antico strato di ghiaccio.

L'antartide invece è un continente ricoperto da uno strato molto spesso di ghiaccio, circondato da ghiaccio in mare (sea ice) e dall'oceano meridionale.

A febbraio 2022 in antartide si è registrata la minore estensione dei ghiacci da quando vengono memorizzati tali dati (dal 1979). Per la prima volta il ghiaccio in acqua ha registrato un'estensione minore di 2 mln di km quadrati 
(https://earthobservatory.nasa.gov/images/149627/antarctic-sea-ice-reaches-record-low)

E' interessante notare che un mese dopo, a marzo 2022, nella regione dell'antartide si sono registrate delle temperature record, come osservato in precedenza con la variabile sst.

In [ ]:
# considerando marzo come fatto in precedenza per la variabile sst

# polo nord

# marzo medio
icec = icec_data.icec.groupby('time.month').mean('time')
icec_mar = icec.isel(month=2)  # 2 corrisponde a marzo

fig = plt.figure(figsize=(10,6))
extent = [180, -180 , 50 , 90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = icec_mar.plot(transform=ccrs.PlateCarree(),
                    vmin=0,
                    vmax=100,
                    cmap=cmocean.cm.ice)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()


# marzo 2022
fig = plt.figure(figsize=(10,6))
extent = [180, -180 , 50 , 90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = icec_data.icec.isel(time=483).plot(transform=ccrs.PlateCarree(),
                                         vmin=0,
                                         vmax=100,
                                         cmap=cmocean.cm.ice)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()

# si nota una diminuzione a marzo 2022 rispetto ai valori tipici del mese di marzo

# nella parte sud-ovest si vede la barriera di Ross o Tavolata di Ross, si tratta della più grande piattaforma glaciale dell'Antartide, 
# con una superficie di circa 473k km^2 e una larghezza di 800 km (ha all'incirca le dimensioni della Francia)

In [ ]:
# polo sud 

# in modo analogo si prende in considerazione il mese di marzo per il polo sud 

# marzo medio
icec = icec_data.icec.groupby('time.month').mean('time')
icec_mar = icec.isel(month=2)  # 2 corrisponde a marzo

fig = plt.figure(figsize=(10,6))
extent = [180, -180 , -50 , -90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=-90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = icec_mar.plot(transform=ccrs.PlateCarree(),
                    vmin=0,
                    vmax=100,
                    cmap=cmocean.cm.ice)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()


# marzo 2022
fig = plt.figure(figsize=(10,6))
extent = [180, -180 , -50 , -90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=-90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = icec_data.icec.isel(time=483).plot(transform=ccrs.PlateCarree(),
                                         vmin=0,
                                         vmax=100,
                                         cmap=cmocean.cm.ice)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()

# si può notare che a marzo 2022 rispetto ai valori medi si ha una diminuzione delle aree in cui vi è ghiaccio

In [ ]:
# focus su febbraio considerando l'antartica

# febbraio medio
icec = icec_data.icec.groupby('time.month').mean('time')
icec_feb = icec.isel(month=1)  # 1 corrisponde a febbraio
fig = plt.figure(figsize=(16,12))
extent = [180, -180 , -50 , -90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=-90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = icec_feb.plot(transform=ccrs.PlateCarree(),
                                                       vmin=0,
                                                       vmax=100,
                                                       cmap=cmocean.cm.ice)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()


# febbraio 2022
fig = plt.figure(figsize=(16,12))
extent = [180, -180 , -50 , -90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=-90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = icec_data.icec.isel(time=482).plot(transform=ccrs.PlateCarree(),
                                                       vmin=0,
                                                       vmax=100,
                                                       cmap=cmocean.cm.ice)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()

# anche in questo caso a febbraio 2022 si nota una diminuzione della presenza di ghiacci in termini percentuali

In [ ]:
# le stagioni ai poli sono solamente due

# polo nord

# artic summer: september
# artic winter: march

icec = icec_data.icec.groupby('time.month').mean('time')  # medie di tutti i mesi per tutti gli anni

# marzo
fig = plt.figure(figsize=(16,12))
extent = [180, -180 , 50 , 90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = icec.isel(month=2).plot(transform=ccrs.PlateCarree(),
                                                       vmin=0,
                                                       vmax=100,
                                                       cmap=cmocean.cm.ice)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()


# settembre
fig = plt.figure(figsize=(16,12))
extent = [180, -180 , 50 , 90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = icec.isel(month=8).plot(transform=ccrs.PlateCarree(),
                                                       vmin=0,
                                                       vmax=100,
                                                       cmap=cmocean.cm.ice)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()

# come si può esservare nella stagione estiva vi è una netta diminuzione della presenza di ghiacci

# durante l'inverno le temperature diminuiscono e si formano i ghiacci (massima estensione a marzo) mentre si sciolgono parzialmente 
# durante l'estate (minima estensione a settembre)

In [ ]:
# considerando le diverse stagioni


# polo sud

# antarctic summer: february
# antarctic winter: september

icec = icec_data.icec.groupby('time.month').mean('time')  # medie di tutti i mesi per tutti gli anni

# settembre
fig = plt.figure(figsize=(16,12))
extent = [180, -180 , -50 , -90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=-90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = icec.isel(month=8).plot(transform=ccrs.PlateCarree(),
                                                       vmin=0,
                                                       vmax=100,
                                                       cmap=cmocean.cm.ice)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()

# febbraio
fig = plt.figure(figsize=(16,12))
extent = [180, -180 , -50 , -90]
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0.0, central_latitude=-90))
ax.set_extent(extent, ccrs.PlateCarree())
p1a = icec.isel(month=1).plot(transform=ccrs.PlateCarree(),
                                                       vmin=0,
                                                       vmax=100,
                                                       cmap=cmocean.cm.ice)
ax.add_feature(cfeature.LAND,zorder=4,color='white')
p1a.axes.coastlines()

# in modo analogo si osserva per il polo sud

## Analisi serie storiche e previsioni <a class="anchor" id="serie-icec"></a>

In [ ]:
plt.figure(figsize=(20, 6))
weights = np.cos(np.deg2rad(icec_data.icec.lat))  # sono i pesi
weights.name = "weights"
icec_weighted = icec_data.icec.weighted(weights)  # la variabile ora pesata, dopo di che posso calcolare la media come prima
weighted_mean = icec_weighted.mean(("lon", "lat"))
weighted_mean.plot()  # serie temporale mensile globale con media pesata

weighted_mean.resample(time="Y").mean().plot()  # serie temporale annuale globale con media pesata

plt.legend(["monthly time series", "annual time series"], loc=0)
plt.show()


# per grafici interattivi
# weights = np.cos(np.deg2rad(icec_data.icec.lat))
# weights.name = "weights"
# icec_weighted = icec_data.icec.weighted(weights)
# weighted_mean = icec_weighted.mean(("lon", "lat"))
# weighted_mean.hvplot()
# weighted_mean.resample(time="Y").mean().hvplot()


# essendo i ghiacciai presenti ai poli questo tipo di informazione in cui si considera la media a livello globale non è del tutto rilevante. 
# Motivo per il quale nelle celle successive si procede ad analizzare l'area specifica nei pressi dei poli

In [ ]:
# senza media pesata
# plt.figure(figsize=(20, 6))
# icec_data.isel(lat=slice(0,24)).mean(("lon", "lat"), keep_attrs=True).icec.groupby("time.month").mean("time").plot()   
# icec_data.isel(lat=slice(156,180)).mean(("lon", "lat"), keep_attrs=True).icec.groupby("time.month").mean("time").plot()  
# plt.legend(["north pole area", "south pole area"], loc=0)
# plt.show()


# con media pesata
plt.figure(figsize=(20, 6))
icec_data.isel(lat=slice(0,24)).icec.weighted(weights).mean(("lon", "lat"), keep_attrs=True).groupby("time.month").mean("time").plot() 
icec_data.isel(lat=slice(156,180)).icec.weighted(weights).mean(("lon", "lat"), keep_attrs=True).groupby("time.month").mean("time").plot()  

plt.legend(["north pole area", "south pole area"], loc=0)
plt.show()

# correttamente si vede che le stagioni sono invertite per i due emisferi

### Polo nord <a class="anchor" id="nord"></a>

In [ ]:
# serie temporale mensile della zona del polo nord

# icec_data.isel(lat=slice(0,24)).mean(("lon", "lat"), keep_attrs=True).icec.hvplot()
icec_data.isel(lat=slice(0,24)).icec.weighted(weights).mean(("lon", "lat"), keep_attrs=True).hvplot()  # con media pesata

# la concentrazione diminuisce tanto nel mese di settembre

# sicuramente si vede un trend negativo che indica che la percentuale dei ghiacciai è diminuita nel tempo

# inoltre si può osservare una varianza maggiore nei mesi estivi rispetto a quanto avveniva negli anni 90

Si utilizza il modello Prophet, un modello sviluppato dall'unità di ricerca di Facebook per trattare serie storiche e fare previsioni

In [ ]:
# preparazione dataset
# icec_data_north = icec_data.isel(lat=slice(0,24)).mean(("lon", "lat"))
icec_data_north = icec_data.isel(lat=slice(0,24)).icec.weighted(weights).mean(("lon", "lat"), keep_attrs=True)  # con media pesata
icec_data_north = icec_data_north.to_dataframe()
icec_data_north = icec_data_north.reset_index()
icec_data_north = icec_data_north[['time', 'icec']]
icec_data_north = icec_data_north.drop_duplicates()
icec_data_north.rename(columns = {'time':'ds', 'icec':'y'}, inplace = True)

# fit del modello
m = Prophet(seasonality_mode='multiplicative').fit(icec_data_north)

# set orizzonte futuro
future = m.make_future_dataframe(periods=120 , freq='M')  # si fanno le previsioni 10 anni in avanti, fino a marzo 2032

# calcolo previsioni
fcst = m.predict(future)

# visualizzazioni risultati
fig = m.plot(fcst)
# si nota che il trend è decisamente negativo come osservato anche in: 
# https://earthobservatory.nasa.gov/images/149627/antarctic-sea-ice-reaches-record-low

a = add_changepoints_to_plot(fig.gca(), m, fcst)
# i changepoints indicano dei punti in cui il trend ha cambiato in modo brusco la sua tendenza. In particolare il trend diventa negativo a 
# partire dagli anni 2003/2004, come confermato anche in https://www.reuters.com/article/us-arctic-ice-idUSTRE56673T20090707 e anche osservato
# nella sezione precedente (Comparazione Northen e Southern Hemisphere Sea Ice Concentration anomaly)

fig = m.plot_components(fcst)
# si osserva la stagionalità mensile e in corrispondenza di settembre come indicato anche in 
# (https://climate.nasa.gov/vital-signs/arctic-sea-ice/) si ha la minima estensione di ghiaccio, come osservato anche dai grafici precedenti

Si procede con una analisi più dettagliata della stagionalità

In [ ]:
# preparazione dati e creazione nuove feature
icec_data_north['ds'] = pd.to_datetime(icec_data_north['ds'])
icec_data_north_ts = icec_data_north.set_index('ds')

icec_data_north_ts['date'] = icec_data_north_ts.index
icec_data_north_ts['month'] = icec_data_north_ts['date'].dt.month
icec_data_north_ts['year'] = icec_data_north_ts['date'].dt.year
icec_data_north_ts

In [ ]:
# visualizzazione stagionalità annuale e mensile
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

sns.boxplot(icec_data_north_ts['year'], icec_data_north_ts['y'], ax=ax[0])
ax[0].set_xlabel('year', fontsize = 14, fontdict=dict(weight='bold'))
ax[0].set_ylabel('y', fontsize = 14, fontdict=dict(weight='bold'))

sns.boxplot(icec_data_north_ts['month'], icec_data_north_ts['y'], ax=ax[1])
ax[1].set_xlabel('month', fontsize = 14, fontdict=dict(weight='bold'))
ax[1].set_ylabel('y', fontsize = 14, fontdict=dict(weight='bold'))


plt.show()
# si nota una chiara stagionalità mensile, come osservato anche in precedenza

In [ ]:
# visualizzazione alternativa per confermare la presenza di stagionalità mensile
plt.figure(figsize=(15, 6))
sns.lineplot(icec_data_north_ts['month'], icec_data_north_ts['y'], hue=icec_data_north_ts['year'], ci=None, palette="plasma")
plt.title('Month plot of Value by year', fontsize = 15, loc='center', fontdict=dict(weight='bold'))
plt.xlabel('Month', fontsize = 12)
plt.ylabel('Value', fontsize = 12)
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
sns.lineplot(icec_data_north_ts['year'], icec_data_north_ts['y'], hue=icec_data_north_ts['month'])
plt.title('Year plot of Value by Month', fontsize = 15, loc='center', fontdict=dict(weight='bold'))
plt.xlabel('Year', fontsize = 12)
plt.ylabel('Value', fontsize = 12)
plt.show()

### Polo sud <a class="anchor" id="sud"></a>

In [ ]:
# icec_data.isel(lat=slice(156,180)).mean(("lon", "lat"), keep_attrs=True).icec.hvplot()
icec_data.isel(lat=slice(156,180)).icec.weighted(weights).mean(("lon", "lat"), keep_attrs=True).hvplot()  # con media pesata

# rispetto al grafico relativo al polo nord ci sono più valori che si avvicinano alla percentuale 100%, indicando una maggiore presenza di 
# ghiaccio

# dal momento in cui l'antartide è circondata completamente da oceani il ghiaccio che si forma in mare nei mesi invernali non viene vincolato 
# nella sua estensione da porzioni di terra ferma come avviene per l'area realativa la polo nord

# durante i mesi estivi si scioglie il ghiaccio che si era formato durante i mesi invernali (da marzo a ottobre). Tuttavia in alcune aree 
# il ghiaccio permane durante tutto l'anno: si tratta ad esempio delle piattaforme di ghiaccio di Ross e di Larsen

# prima i picchi negativi erano su settembre, ora su febbraio

Si utilizza il modello Prophet, un modello sviluppato dall'unità di ricerca di Facebook per trattare serie storiche e fare previsioni

In [ ]:
# preparazione dataset
# icec_data_south = icec_data.isel(lat=slice(156,180)).mean(("lon", "lat"))
icec_data_south = icec_data.isel(lat=slice(156,180)).icec.weighted(weights).mean(("lon", "lat"), keep_attrs=True)  # con media pesata
icec_data_south = icec_data_south.to_dataframe()
icec_data_south = icec_data_south.reset_index()
icec_data_south = icec_data_south[['time', 'icec']]
icec_data_south = icec_data_south.drop_duplicates()
icec_data_south.rename(columns = {'time':'ds', 'icec':'y'}, inplace = True)

# fit modello
m = Prophet(seasonality_mode='multiplicative').fit(icec_data_south)

# set orizzonte temporale
future = m.make_future_dataframe(periods=120 , freq='M')  # si fanno le previsioni 10 anni in avanti, fino a marzo 2032

# calcolo previsioni
fcst = m.predict(future)

# visualizzazione risultati
fig = m.plot(fcst)
# anche in questo caso il trend è negativo per gli anni futuri, anche se in misura minore rispetto al caso precedente

a = add_changepoints_to_plot(fig.gca(), m, fcst)
# vengono indicati come change points gli anni prima del 2014. Proprio a partire dal 2014 l'estensione dei ghiacci in antartica ha iniziato 
# ad andare incontro ad un declino raggiungendo un minimo storico nel 2017
# https://royalsociety.org/topics-policy/projects/climate-change-evidence-causes/question-12/
# https://www.pnas.org/doi/10.1073/pnas.1906556116

fig = m.plot_components(fcst)
# in questo caso in marzo si misurano i valori più bassi

Si procede con una analisi più dettagliata della stagionalità

In [ ]:
# preparazione dati e creazione nuove feature
icec_data_south['ds'] = pd.to_datetime(icec_data_south['ds'])
icec_data_south_ts = icec_data_south.set_index('ds')

icec_data_south_ts['date'] = icec_data_south_ts.index
icec_data_south_ts['month'] = icec_data_south_ts['date'].dt.month
icec_data_south_ts['year'] = icec_data_south_ts['date'].dt.year

icec_data_south_ts

In [ ]:
# visualizzazione stagionalità annuale e mensile
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))

sns.boxplot(icec_data_south_ts['year'], icec_data_south_ts['y'], ax=ax[0])
ax[0].set_xlabel('year', fontsize = 14, fontdict=dict(weight='bold'))
ax[0].set_ylabel('y', fontsize = 14, fontdict=dict(weight='bold'))

sns.boxplot(icec_data_south_ts['month'], icec_data_south_ts['y'], ax=ax[1])
ax[1].set_xlabel('month', fontsize = 14, fontdict=dict(weight='bold'))
ax[1].set_ylabel('y', fontsize = 14, fontdict=dict(weight='bold'))

plt.show()

# si nota una chiara stagionalità mensile, come osservato in precedenza

In [ ]:
# visualizzazione alternativa per confermare la presenza di stagionalità mensile
plt.figure(figsize=(15, 6))
sns.lineplot(icec_data_south_ts['month'], icec_data_south_ts['y'], hue=icec_data_south_ts['year'], ci=None, palette="plasma")
plt.title('Month plot of Value by year', fontsize = 15, loc='center', fontdict=dict(weight='bold'))
plt.xlabel('Month', fontsize = 12)
plt.ylabel('Value', fontsize = 12)
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
sns.lineplot(icec_data_south_ts['year'], icec_data_south_ts['y'], hue=icec_data_south_ts['month'])
plt.title('Year plot of Value by Month', fontsize = 15, loc='center', fontdict=dict(weight='bold'))
plt.xlabel('Year', fontsize = 12)
plt.ylabel('Value', fontsize = 12)
plt.show()

# Sea Surface Temperature data e Ice Concentration data <a class="anchor" id="correlation-data"></a>

## Global Sea Surface Temperature e Ice Concentration Anomaly <a class="anchor" id="global"></a>

In [ ]:
# anomalie sst

fig,ax = plt.subplots(figsize=(10,7))
# ds = sst_data.sst.mean(dim=['lat','lon'])  # per ogni istante temporale si ha un valore di sst che è la media delle sst di tutte le celle 
# in quell'istante temporale
ds = sst_data.sst.weighted(weights).mean(dim=['lat','lon'])  # media pesata
enso_clim = ds.sel(time=slice('1982-01-01','2010-12-01')).groupby("time.month").mean("time")  # sono le normali climatiche, per ogni mese si ha una media, 
# quindi si avrà ad esempio gennaio che è la media di tutti i gennai di tutti gli anni che si hanno a disposizione
# ds.groupby("time.month"): sono le osservazioni, si avranno X gennai, X febbrai tutti di anni diversi
enso_anom_sst = ds.groupby("time.month") - enso_clim
enso_anom_sst.plot(color="black")
ax.axhline(y=0, c='black', linewidth=1, linestyle='dashdot')
plt.title('Sea Surface Temperature anomaly')

# effettivamente c'è stato sempre più un aumento di temperaturatura, delle anomalie positive in particolare negli ultimi anni
# si registra una maggiore variabilità



# anomalie sea ice

fig,ax = plt.subplots(figsize=(10,7))
# ds = icec_data.icec.mean(dim=['lat','lon'])
ds = icec_data.icec.weighted(weights).mean(dim=['lat','lon'])  # media pesata
enso_clim = ds.sel(time=slice('1982-01-01','2010-12-01')).groupby("time.month").mean("time")
enso_anom_ice = ds.groupby("time.month") - enso_clim
enso_anom_ice.plot(color="black")
ax.axhline(y=0, c='black', linewidth=1, linestyle='dashdot')

# effettivamente negli ultimi anni c'è stata una diminuzione di sea ice, tra il 2016 e il 2019 in particolare vi sono anomalie nettamente negative

## Correlazione tra Sea Surface Temperature e Ice Concentration <a class="anchor" id="correlation"></a>

In [ ]:
# concentrandosi su una piccola porzione presa in modo arbitrario sarebbe interessante osservare e avere la conferma che ad un aumento della 
# temperatura corrisponde una diminuzione delle percentuali di ghiaccio

icec_data.icec.groupby("time.month").mean("time")[:,2,80].hvplot()
# si vede nel dettaglio ciò che è stato notato sopra, al polo nord nel mese di agosto settembre vi è la minore concentrazione di ghiaccio nelle acque

In [ ]:
sst_data.sst.groupby("time.month").mean("time")[:,2,80].hvplot()

# effettivamente proprio ad agosto/settembre si nota un aumento seppur impercettibile delle temperature

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
sns.regplot(sst_data.sst.weighted(weights).mean(dim=['lat', 'lon']),
            icec_data.icec.weighted(weights).mean(dim=['lat', 'lon']), 
            data=icec_data.icec.weighted(weights).mean(dim=['lat', 'lon']), 
            fit_reg=True,
            line_kws={"linewidth": 3, "color": "tomato"})
ax.grid(linewidth=0.5)
ax.set_ylabel('Sea Ice Concentration %')
ax.set_xlabel('Sea Surface Temperature °C')
ax.set_title('Correlation of Sea Ice Concentration and Sea Surface Temperature')
plt.tight_layout()

# osservando il grafico ottenuto la sea ice concentration diminuisce all'aumentare della sea surface temperature. E' possibile dunque concludere
# che vi sia una correlazione negativa tra sea surface temperature e sea ice concentration

In [ ]:
# si calcola l'indice di correlazione di Pearson
corr, _ = pearsonr(sst_data.sst.weighted(weights).mean(dim=['lat', 'lon']), 
                   icec_data.icec.weighted(weights).mean(dim=['lat', 'lon']))
print('Pearsons correlation: %.3f' % corr)
# come previsto si ottiene un valore negativo